In [ ]:
import requests
import pandas as pd
import openpyxl

# Переменные
first_name = 'Индексы_видов_деятельности_по_ОКВЭД2_в_Российской_Федерации_за_2023г'
second_name = ('Топ_10_индексов_производства_по_отдельным_видам_экономической_деятельности_'
               'по_Российской_Федерации_за_2023г')
data_excel = []
headers = []

In [ ]:
# Скачиваем файл для дальнейшего парсинга
response = requests.get('https://rosstat.gov.ru/storage/mediabank/ind_god_2015-2022.xlsx')
with open('data.xlsx', 'wb') as file:
    file.write(response.content)

# Открываем файл excell
workbook = openpyxl.load_workbook('data.xlsx')
sheet = workbook['1']
row_number = 4
row_data = sheet[row_number]

# Парсим хидеры из файла
for cell in row_data:
    if cell.value is None:
        pass
    else:
        headers.append(cell.value)

# Парсим все значения внутри таблицы
for row in sheet.iter_rows(min_row=5, max_row=135, values_only=True):
    data_row = []
    for cell in row:
        if cell is not None:
            data_row.append(cell)
    data_excel.append(data_row)

# Редактируем название хидеров
headers[-1] = '2023'
headers[9] = '2021 Второе полугодие'
data = pd.DataFrame(data_excel, columns=headers)

# Паттерн для создания необходимого нам даташита
pattern = ((data['Код ОКВЭД2'] == 'B') |
           (data['Код ОКВЭД2'] == 'C') |
           (data['Код ОКВЭД2'] == 'D') |
           (data['Код ОКВЭД2'] == 'E') |
           (data['Код ОКВЭД2'] == 'BCDE'))
pattern_headers = ['Наименование вида деятельности по ОКВЭД2', 'Код ОКВЭД2', '2023']

# Фильтруем записи по нашему паттерну
filter_by_first = data[pattern]
filter_by_second = data.loc[~pattern]

filtered_one = filter_by_first[pattern_headers]
filtered_two = filter_by_second.sort_values(by=['2023'], ascending=False)[pattern_headers]

# Выгружаем даташиты в csv для дальнейших манипуляций
filtered_one.to_csv(first_name, index=False)
filtered_two[:10].to_csv(second_name, index=False)

workbook.close()


In [ ]:
file = pd.read_csv(first_name)
print(file)

In [ ]:
file = pd.read_csv(second_name)
print(file)